In [1]:
# Write down all paths.


PATH1 = r"C:\Users\enesy\Videos\xd\1.mp4"
PATH2 = r"C:\Users\enesy\Videos\xd\2.MOV"
PATH3 = r"C:\Users\enesy\Videos\xd\3.mp4"
PATH4 = r"C:\Users\enesy\Videos\xd\4.MP4"

In [2]:
# Find all delays
# Credit: Bucky und Stef

from prototype import *

all_paths = [PATH1, PATH2, PATH3, PATH4]

delays = find_all_delays(all_paths)

delays

pygame 2.6.1 (SDL 2.28.4, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'avc1', 'minor_version': '0', 'compatible_brands': 'avc1isom', 'creation_time': '2024-11-16T09:45:49.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [1920, 1080], 'bitrate': 23988, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'rotate': 180.0, 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AVC', 'encoder': 'Ambarella AVC encoder', 'Side data': '', 'displaymatrix': 'rotation of -180.00 degrees'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'rotate': '180', 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AAC'}}], 'input_number': 0}], 'dura

[0.0, 18.829319727891157, 0.3985487528344671, 12.75968253968254]

In [3]:
# We need to know the durations of videos
# We will update these as we go.

from moviepy import VideoFileClip

durations = [VideoFileClip(path).duration for path in all_paths]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'avc1', 'minor_version': '0', 'compatible_brands': 'avc1isom', 'creation_time': '2024-11-16T09:45:49.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [1920, 1080], 'bitrate': 23988, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'rotate': 180.0, 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AVC', 'encoder': 'Ambarella AVC encoder', 'Side data': '', 'displaymatrix': 'rotation of -180.00 degrees'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'rotate': '180', 'creation_time': '2024-11-16T09:45:49.000000Z', 'handler_name': 'Ambarella AAC'}}], 'input_number': 0}], 'duration': 415.72, 'bitrate': 24234, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0,

In [21]:
# This is the edited commmand to include audio

import ffmpeg

OUTPUT_FILE_NAME_WITH_EXTENSION = "split_xstack_test.mp4"

# Load videos into ffmpeg
videos = [ffmpeg.input(path) for path in all_paths]

video_streams = [video.video for video in videos]

# Handling audio seperately
audio_streams = [video.audio for video in videos]

# We handle each video individually.
for index, video in enumerate(video_streams):

    # If the delay of the vide is negative:
    if delays[index] < 0:
        # We trim the video from the start
        video_streams[index] = ffmpeg.trim(video, start=round(delays[index], 2)).setpts("PTS-STARTPTS")
        
        # Update the corresponding video's duration
        durations[index] -= round(delays[index])

    else: # If the delay is positive
        # We delay/push the videos start by the amount
        video_streams[index] = ffmpeg.filter(video, "tpad", start_duration=round(delays[index], 2))

        audio_streams[index] = ffmpeg.filter(audio_streams[index], "apad", pad_dur=round(delays[index], 2))
        
# Put the videos on grid 2x2
xstacked = ffmpeg.filter(video_streams, "xstack", inputs="4", layout="0_0|0_h0|w0_0|w0_h0")

# Trying to split xstack

# We cut the whole video by the longest positive delay
# So that the videos begin at the same time.
video_start_adjusted = ffmpeg.trim(xstacked, start=max(delays)).setpts("PTS-STARTPTS")

# We take the audio stream with the maximum delay.
maximum_delay = np.argmax(delays)
audio_stream = audio_streams[maximum_delay]

# Cut the start of audio similarly
# This works because there will always be at least one 0 or larger delay.
# Thus no need to account for negatives.
audios_start_adjusted = ffmpeg.filter(audio_stream, "atrim", start=max(delays)).filter("asetpts", "PTS-STARTPTS")

# Scale down the video to 1080p 60FPS
# Normally, we are supposed to output 4K 60FPS
video_scaled = ffmpeg.filter(video_start_adjusted, "scale", 3840, 2160).filter('fps', fps=60, round='up')

# The output uses ultrafast preset for demonstration purposes.
# The video will continue until the shortest video ends, i.e set stop_duration = min(durations)
# If you want to test it out for a shorter time, change the "stop_duration" to amount of seconds you want (e.g 10 secs)
stop_duration = min(durations)
out = ffmpeg.output(audio_stream, video_scaled, OUTPUT_FILE_NAME_WITH_EXTENSION, acodec="aac", vcodec="libx264", pix_fmt="yuv420p", crf=21, preset="fast", profile="high", progress="pipe:1", to=stop_duration)

# Compile command will return the terminal command to run FFMPEG as a string
command = out.compile()

print(" ".join(command))

ffmpeg -i C:\Users\enesy\Videos\xd\2.MOV -i C:\Users\enesy\Videos\xd\1.mp4 -i C:\Users\enesy\Videos\xd\3.mp4 -i C:\Users\enesy\Videos\xd\4.MP4 -filter_complex [0:a]apad=pad_dur=18.83[s0];[1:v]tpad=start_duration=0.0[s1];[0:v]tpad=start_duration=18.83[s2];[2:v]tpad=start_duration=0.4[s3];[3:v]tpad=start_duration=12.76[s4];[s1][s2][s3][s4]xstack=inputs=4:layout=0_0|0_h0|w0_0|w0_h0[s5];[s5]trim=start=18.829319727891157[s6];[s6]setpts=PTS-STARTPTS[s7];[s7]scale=3840:2160[s8] -map [s0] -map [s8] -acodec aac -crf 21 -pix_fmt yuv420p -preset fast -profile high -progress pipe:1 -to 5 -vcodec libx264 split_xstack_test.mp4


In [22]:
# We are going to use pipes and subprocess
# This is because we want to see the progress of FFMPEG from our main process (i.e the notebook)

import subprocess
import datetime # For printing progress

# Run the FFMPEG subprocess
try:
    ffmpeg_process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
except OSError:
    print("OSError")
except ValueError:
    print("ValueError: Cannot run FFMPEG with given parameters.")

end_time = str(datetime.timedelta(seconds=stop_duration)) # Used for printing the last point of the video
# Print the progress that ffmpeg outputs
print("FFMPEG is starting, please wait a couple minutes.")
for line in ffmpeg_process.stdout:
    # for all output from ffmpeg
    print(line)

    # If you would like to print it in a simpler format
    # This shows how much of the video you processed
    # if line.startswith("out_time="):
    #     progress = line.split("=")[1]
    #     print(f"Progress: currently at {progress} / going until {end_time}", end="\r")
print("FFMPEG has finished processing.")

# IMPORTANT: you have to quit the FFMPEG process itself from Task Manager to stop it
# It will also not continue if the file exists already, make sure to delete the file if it looks like its doing nothing.


FFMPEG is starting, please wait a couple minutes.
ffmpeg version 2024-05-08-git-e8e84dc325-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers

  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)

  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r -